# Dataset Comparison: Competition vs Original Hill of Towie (2016-2020)

## Summary

This notebook provides a comparison between:
1. **Competition Dataset**: Pre-processed Kaggle competition files (2016-2020)
2. **Original Dataset**: Raw Hill of Towie data from Zenodo (2016-2020)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
from datetime import datetime, timedelta
import warnings
from typing import Dict, List, Tuple, Optional
import os

warnings.filterwarnings('ignore')

# Configure visualization
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Setup paths
PROJECT_ROOT = Path('../').resolve()
DATA_DIR = PROJECT_ROOT / 'data'
ORIGINAL_DATA_DIR = DATA_DIR / 'external' / 'hill-of-towie-original'
EXTRACT_DIR = ORIGINAL_DATA_DIR / 'extracted'
REPO_DIR = DATA_DIR / 'external' / 'hill-of-towie-repo'

# Competition data paths
TRAIN_PATH = DATA_DIR / 'train' / 'training_dataset.parquet'
TEST_PATH = DATA_DIR / 'test' / 'submission_dataset.parquet'
SAMPLE_SUB_PATH = DATA_DIR / 'output' / 'sample_model_submission.csv'

print("📁 Path Configuration:")
print(f"   Project root: {PROJECT_ROOT}")
print(f"   Competition data: {DATA_DIR}")
print(f"   Original data: {ORIGINAL_DATA_DIR}")
print(f"   Extracted data: {EXTRACT_DIR}")
print(f"\n✅ Path exists check:")
print(f"   Training data: {TRAIN_PATH.exists()}")
print(f"   Test data: {TEST_PATH.exists()}")
print(f"   Original data dir: {ORIGINAL_DATA_DIR.exists()}")
print(f"   Extracted data: {EXTRACT_DIR.exists()}")

📁 Path Configuration:
   Project root: /home/boujuan/Coding/hill-of-towie-wind-turbine
   Competition data: /home/boujuan/Coding/hill-of-towie-wind-turbine/data
   Original data: /home/boujuan/Coding/hill-of-towie-wind-turbine/data/external/hill-of-towie-original
   Extracted data: /home/boujuan/Coding/hill-of-towie-wind-turbine/data/external/hill-of-towie-original/extracted

✅ Path exists check:
   Training data: True
   Test data: True
   Original data dir: True
   Extracted data: True


## 1. Competition Dataset Analysis

In [2]:
# Load competition data
train_df = pd.read_parquet(TRAIN_PATH)
test_df = pd.read_parquet(TEST_PATH)
sample_sub = pd.read_csv(SAMPLE_SUB_PATH)

print("🏆 COMPETITION DATASET OVERVIEW")
print("="*80)

# Basic statistics
print(f"\n📊 Dataset Shapes:")
print(f"   Training: {train_df.shape} ({train_df.shape[0]:,} rows × {train_df.shape[1]} columns)")
print(f"   Test: {test_df.shape} ({test_df.shape[0]:,} rows × {test_df.shape[1]} columns)")
print(f"   Sample submission: {sample_sub.shape}")

# Temporal coverage
print(f"\n📅 Temporal Coverage:")
print(f"   Training period: {train_df['TimeStamp_StartFormat'].min()} to {train_df['TimeStamp_StartFormat'].max()}")
print(f"   Test period: {test_df['TimeStamp_StartFormat'].min()} to {test_df['TimeStamp_StartFormat'].max()}")

# Calculate actual time spans
train_days = (train_df['TimeStamp_StartFormat'].max() - train_df['TimeStamp_StartFormat'].min()).days
test_days = (test_df['TimeStamp_StartFormat'].max() - test_df['TimeStamp_StartFormat'].min()).days
print(f"   Training duration: {train_days} days (~{train_days/365:.1f} years)")
print(f"   Test duration: {test_days} days (~{test_days/365:.1f} years)")

# Data frequency
time_diffs = train_df['TimeStamp_StartFormat'].diff().dropna()
print(f"\n⏱️ Data Frequency:")
print(f"   Sampling interval: {time_diffs.mode()[0]}")
print(f"   Records per day: {24 * 60 / 10:.0f}")
print(f"   Expected annual records: {365 * 24 * 6:,}")

🏆 COMPETITION DATASET OVERVIEW

📊 Dataset Shapes:
   Training: (210384, 189) (210,384 rows × 189 columns)
   Test: (52704, 159) (52,704 rows × 159 columns)
   Sample submission: (52704, 2)

📅 Temporal Coverage:
   Training period: 2016-01-01 00:00:00+00:00 to 2019-12-31 23:50:00+00:00
   Test period: 2020-01-01 00:00:00+00:00 to 2020-12-31 23:50:00+00:00
   Training duration: 1460 days (~4.0 years)
   Test duration: 365 days (~1.0 years)

⏱️ Data Frequency:
   Sampling interval: 0 days 00:10:00
   Records per day: 144
   Expected annual records: 52,560


In [3]:
# Analyze turbine coverage
print("🌬️ TURBINE ANALYSIS")
print("="*80)

# Extract turbine information from column names
turbine_cols = [col for col in train_df.columns if ';' in col]
turbine_fields = {}

for col in turbine_cols:
    parts = col.split(';')
    if len(parts) == 2 and parts[1].isdigit():
        field, turbine = parts
        turbine_id = int(turbine)
        if turbine_id not in turbine_fields:
            turbine_fields[turbine_id] = []
        turbine_fields[turbine_id].append(field)

competition_turbines = sorted(turbine_fields.keys())
print(f"\n🔧 Turbines in Competition Data:")
print(f"   Available: {competition_turbines}")
print(f"   Missing: Turbine 6 (gap between 5 and 7)")
print(f"   Total: {len(competition_turbines)} turbines")

# Fields per turbine
if competition_turbines:
    sample_turbine = competition_turbines[0]
    fields = turbine_fields[sample_turbine]
    print(f"\n📋 Fields per Turbine: {len(fields)}")
    print("\n   SCADA Fields:")
    scada_fields = [f for f in fields if not f.startswith('ERA5_') and f != 'ShutdownDuration']
    for i, field in enumerate(scada_fields[:10], 1):
        print(f"     {i:2}. {field}")
    if len(scada_fields) > 10:
        print(f"     ... and {len(scada_fields)-10} more SCADA fields")
    
    print("\n   Weather Fields (ERA5):")
    weather_fields = [f for f in fields if f.startswith('ERA5_')]
    for i, field in enumerate(weather_fields[:5], 1):
        print(f"     {i:2}. {field}")
    if len(weather_fields) > 5:
        print(f"     ... and {len(weather_fields)-5} more ERA5 fields")

🌬️ TURBINE ANALYSIS

🔧 Turbines in Competition Data:
   Available: [1, 2, 3, 4, 5, 7]
   Missing: Turbine 6 (gap between 5 and 7)
   Total: 6 turbines

📋 Fields per Turbine: 29

   SCADA Fields:
      1. wtc_AcWindSp_mean
      2. wtc_AcWindSp_min
      3. wtc_AcWindSp_max
      4. wtc_AcWindSp_stddev
      5. wtc_ScYawPos_mean
      6. wtc_ScYawPos_min
      7. wtc_ScYawPos_max
      8. wtc_ScYawPos_stddev
      9. wtc_NacelPos_mean
     10. wtc_NacelPos_min
     ... and 18 more SCADA fields

   Weather Fields (ERA5):


In [4]:
# Analyze target and validation
print("🎯 TARGET & VALIDATION ANALYSIS")
print("="*80)

# Target column
print(f"\n📊 Target Variable:")
print(f"   Column name: 'target'")
print(f"   Definition: Active power of Turbine 1 (clipped at 0)")
print(f"   Statistics:")
print(f"     - Mean: {train_df['target'].mean():.2f} kW")
print(f"     - Std: {train_df['target'].std():.2f} kW")
print(f"     - Min: {train_df['target'].min():.2f} kW")
print(f"     - Max: {train_df['target'].max():.2f} kW")
print(f"     - Zero values: {(train_df['target'] == 0).sum():,} ({(train_df['target'] == 0).mean()*100:.1f}%)")

# Validation flag
print(f"\n✅ Validation Flag (is_valid):")
print(f"   Purpose: Only valid periods count for competition scoring")
print(f"   Valid records: {train_df['is_valid'].sum():,} ({train_df['is_valid'].mean()*100:.1f}%)")
print(f"   Invalid records: {(~train_df['is_valid']).sum():,} ({(~train_df['is_valid']).mean()*100:.1f}%)")

# is_valid conditions
print(f"\n   Validity Conditions (all must be true):")
print(f"     1. ShutdownDuration;1 == 0 (turbine not shut down)")
print(f"     2. wtc_ScReToOp_timeon;1 == 600 (full 10-min operation)")
print(f"     3. wtc_ActPower_mean;1 is not null")

# Check validity conditions
if 'ShutdownDuration;1' in train_df.columns:
    shutdown_zero = (train_df['ShutdownDuration;1'] == 0).sum()
    full_operation = (train_df['wtc_ScReToOp_timeon;1'] == 600).sum()
    power_not_null = train_df['wtc_ActPower_mean;1'].notna().sum()
    
    print(f"\n   Condition Breakdown:")
    print(f"     - No shutdown: {shutdown_zero:,} records")
    print(f"     - Full operation: {full_operation:,} records")
    print(f"     - Power not null: {power_not_null:,} records")

🎯 TARGET & VALIDATION ANALYSIS

📊 Target Variable:
   Column name: 'target'
   Definition: Active power of Turbine 1 (clipped at 0)
   Statistics:
     - Mean: 644.15 kW
     - Std: 713.70 kW
     - Min: 0.00 kW
     - Max: 2304.95 kW
     - Zero values: 32,328 (15.4%)

✅ Validation Flag (is_valid):
   Purpose: Only valid periods count for competition scoring
   Valid records: 201,323 (95.7%)
   Invalid records: 9,061 (4.3%)

   Validity Conditions (all must be true):
     1. ShutdownDuration;1 == 0 (turbine not shut down)
     2. wtc_ScReToOp_timeon;1 == 600 (full 10-min operation)
     3. wtc_ActPower_mean;1 is not null

   Condition Breakdown:
     - No shutdown: 202,959 records
     - Full operation: 201,386 records
     - Power not null: 207,924 records


## 2. Original Dataset Analysis

In [5]:
print("📦 ORIGINAL DATASET OVERVIEW")
print("="*80)

# Check download status
available_years = []
year_info = {}

if EXTRACT_DIR.exists():
    year_dirs = sorted([d for d in EXTRACT_DIR.iterdir() if d.is_dir()])
    available_years = [d.name for d in year_dirs]
    
    print(f"\n📅 Years Available: {', '.join(available_years) if available_years else 'None - download in progress'}")
    
    # Analyze each year
    for year_dir in year_dirs:
        year = year_dir.name
        csv_files = list(year_dir.glob('*.csv'))
        total_size = sum(f.stat().st_size for f in csv_files) / (1024**3)  # GB
        
        # Group by table type
        tables = {}
        for f in csv_files:
            table_name = f.name.split('_')[0] if '_' in f.name else f.stem
            if table_name not in tables:
                tables[table_name] = []
            tables[table_name].append(f)
        
        year_info[year] = {
            'files': len(csv_files),
            'size_gb': total_size,
            'tables': list(tables.keys())
        }
        
        print(f"\n   📁 Year {year}:")
        print(f"      Files: {len(csv_files)}")
        print(f"      Size: {total_size:.2f} GB")
        print(f"      Tables: {', '.join(tables.keys())}")
else:
    print("\n⚠️ Original data not yet extracted. Run download_original_data.py first.")

# Check GitHub repo for additional context
if REPO_DIR.exists():
    print(f"\n📚 GitHub Repository: ✅ Available at {REPO_DIR}")
    # Check for useful scripts
    scripts_dir = REPO_DIR / 'scripts'
    if scripts_dir.exists():
        py_files = list(scripts_dir.rglob('*.py'))
        print(f"   Processing scripts found: {len(py_files)}")
else:
    print(f"\n📚 GitHub Repository: ❌ Not cloned")

📦 ORIGINAL DATASET OVERVIEW

📅 Years Available: 2016, 2017, 2018, 2019, 2020

   📁 Year 2016:
      Files: 156
      Size: 6.27 GB
      Tables: tblAlarmLog, tblDailySummary, tblGridScientific, tblGrid, tblSCTurbine, tblSCTurCount, tblSCTurDigiIn, tblSCTurDigiOut, tblSCTurFlag, tblSCTurGrid, tblSCTurIntern, tblSCTurPress, tblSCTurTemp

   📁 Year 2017:
      Files: 156
      Size: 5.49 GB
      Tables: tblAlarmLog, tblDailySummary, tblGridScientific, tblGrid, tblSCTurbine, tblSCTurCount, tblSCTurDigiIn, tblSCTurDigiOut, tblSCTurFlag, tblSCTurGrid, tblSCTurIntern, tblSCTurPress, tblSCTurTemp

   📁 Year 2018:
      Files: 156
      Size: 5.59 GB
      Tables: tblAlarmLog, tblDailySummary, tblGridScientific, tblGrid, tblSCTurbine, tblSCTurCount, tblSCTurDigiIn, tblSCTurDigiOut, tblSCTurFlag, tblSCTurGrid, tblSCTurIntern, tblSCTurPress, tblSCTurTemp

   📁 Year 2019:
      Files: 156
      Size: 5.58 GB
      Tables: tblAlarmLog, tblDailySummary, tblGridScientific, tblGrid, tblSCTurbine, tbl

In [6]:
# Deep dive into data structure if available
if available_years and '2016' in available_years:
    print("🔍 ORIGINAL DATA STRUCTURE ANALYSIS (2016 Sample)")
    print("="*80)
    
    year_dir = EXTRACT_DIR / '2016'
    
    # Analyze each table type
    table_samples = {}
    
    for table_type in ['tblSCTurbine', 'tblSCTurGrid', 'tblSCTurTemp', 'tblSCTurFlag']:
        files = list(year_dir.glob(f'{table_type}*.csv'))
        if files:
            print(f"\n📊 {table_type}:")
            # Read sample
            df_sample = pd.read_csv(files[0], nrows=1000)
            table_samples[table_type] = df_sample
            
            print(f"   Shape: {df_sample.shape}")
            print(f"   Columns: {len(df_sample.columns)}")
            
            # Check for StationId (turbine identifier)
            if 'StationId' in df_sample.columns:
                unique_stations = df_sample['StationId'].unique()
                turbine_ids = sorted([sid - 2304509 for sid in unique_stations])
                print(f"   Turbines in sample: {turbine_ids[:10]}..." if len(turbine_ids) > 10 else f"   Turbines: {turbine_ids}")
                
                # Check for Turbine 6
                turbine_6_station = 2304515
                has_turbine_6 = turbine_6_station in unique_stations
                print(f"   🎯 Turbine 6 (StationId {turbine_6_station}): {'✅ Present' if has_turbine_6 else '❌ Not in sample'}")
            
            # Show key columns
            if table_type == 'tblSCTurbine':
                key_cols = ['wtc_AcWindSp_mean', 'wtc_ScYawPos_mean', 'wtc_NacelPos_mean', 
                           'wtc_GenRpm_mean', 'wtc_PitcPosA_mean', 'wtc_PitcPosB_mean', 'wtc_PitcPosC_mean']
                print("   Key columns available:")
                for col in key_cols:
                    if col in df_sample.columns:
                        print(f"     ✓ {col}")
                    else:
                        print(f"     ✗ {col}")
            
            elif table_type == 'tblSCTurGrid':
                if 'wtc_ActPower_mean' in df_sample.columns:
                    print(f"   ✅ Contains active power (target variable source)")
else:
    print("\n⚠️ Cannot analyze structure - data not yet available")

🔍 ORIGINAL DATA STRUCTURE ANALYSIS (2016 Sample)

📊 tblSCTurbine:
   Shape: (1000, 85)
   Columns: 85
   Turbines in sample: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]...
   🎯 Turbine 6 (StationId 2304515): ✅ Present
   Key columns available:
     ✓ wtc_AcWindSp_mean
     ✓ wtc_ScYawPos_mean
     ✓ wtc_NacelPos_mean
     ✓ wtc_GenRpm_mean
     ✓ wtc_PitcPosA_mean
     ✓ wtc_PitcPosB_mean
     ✓ wtc_PitcPosC_mean

📊 tblSCTurGrid:
   Shape: (1000, 52)
   Columns: 52
   Turbines in sample: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.int64(9), np.int64(10)]...
   🎯 Turbine 6 (StationId 2304515): ✅ Present
   ✅ Contains active power (target variable source)

📊 tblSCTurTemp:
   Shape: (1000, 142)
   Columns: 142
   Turbines in sample: [np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8), np.

## 3. Feature Comparison

In [7]:
print("📊 FEATURE COMPARISON")
print("="*80)

# Competition features (excluding turbine ID)
comp_features = [col.split(';')[0] for col in train_df.columns if ';1' in col]
comp_features_unique = list(dict.fromkeys(comp_features))  # Preserve order, remove duplicates

print(f"\n🏆 Competition Dataset Features:")
print(f"   Total unique features per turbine: {len(comp_features_unique)}")

# Categorize features
scada_features = [f for f in comp_features_unique if not f.startswith('ERA5_') and f != 'ShutdownDuration']
weather_features = [f for f in comp_features_unique if f.startswith('ERA5_')]
operational_features = ['ShutdownDuration'] if 'ShutdownDuration' in comp_features_unique else []

print(f"\n   Feature Categories:")
print(f"     SCADA: {len(scada_features)} features")
print(f"     Weather (ERA5): {len(weather_features)} features")
print(f"     Operational: {len(operational_features)} features")

# If original data is available, compare
if available_years and 'table_samples' in locals():
    print(f"\n📦 Original Dataset Features:")
    
    all_original_cols = set()
    for table_name, df in table_samples.items():
        cols = set(df.columns) - {'TimeStamp', 'StationId'}
        all_original_cols.update(cols)
    
    print(f"   Total unique features: {len(all_original_cols)}")
    
    # Find matching features
    matches = []
    for comp_feat in scada_features:
        for orig_feat in all_original_cols:
            if comp_feat in orig_feat or orig_feat in comp_feat:
                matches.append((comp_feat, orig_feat))
                break
    
    print(f"\n   Feature Overlap:")
    print(f"     Matching features: {len(matches)}/{len(scada_features)}")
    print(f"     Coverage: {len(matches)/len(scada_features)*100:.1f}%")
    print(f"     Additional original features: {len(all_original_cols) - len(matches)}")
    
    # Show unique original features
    print(f"\n   Unique to Original (sample):")
    unique_original = all_original_cols - set([m[1] for m in matches])
    for i, feat in enumerate(list(unique_original)[:10], 1):
        print(f"     {i:2}. {feat}")
    if len(unique_original) > 10:
        print(f"     ... and {len(unique_original)-10} more")

📊 FEATURE COMPARISON

🏆 Competition Dataset Features:
   Total unique features per turbine: 29

   Feature Categories:
     SCADA: 28 features
     Weather (ERA5): 0 features
     Operational: 1 features

📦 Original Dataset Features:
   Total unique features: 317

   Feature Overlap:
     Matching features: 28/28
     Coverage: 100.0%
     Additional original features: 289

   Unique to Original (sample):
      1. wtc_HydOilTm_max
      2. wtc_TowerFrq_Frequenc_max
      3. wtc_A3LefTmp_stddev
      4. wtc_IMSRotTm_stddev
      5. wtc_HSGenTmp_max
      6. wtc_PitcRefA_stddev
      7. wtc_TetAnFrq_stddev
      8. wtc_RawPower_endvalue
      9. wtc_GeOilTmp_max
     10. wtc_ScInOper_counts
     ... and 279 more
